# Calibration analysis (Black / Heston / SVCJ) — thesis figures & tables

This notebook turns `calibration_results.xlsx` into the figures and tables used in the thesis calibration-results section.

**Design choices**
- **Snapshot is the sampling unit**. Where we report confidence intervals for average errors, we bootstrap **over snapshots** (not over individual quotes).
- Errors are measured in **coin premium units** (inverse option numeraire).
- We report both **price-space errors** (RMSE/MAE) and **microstructure-aware** diagnostics (within-spread fractions, error/spread, etc.).

> If you moved the Excel file, update `DATA_PATH` in the next cell.


In [1]:

# --- imports & settings ---
import os
import numpy as np
import pandas as pd

from IPython.display import display

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 180)

# Plotly defaults
pio.templates.default = "plotly_white"
# Renderer (works well in Jupyter)
pio.renderers.default = "plotly_mimetype"

# Reproducibility (bootstrap)
RNG = np.random.default_rng(123)

# Paths
DATA_PATH = "calibration_results.xlsx"  # <-- change if needed

# Models and columns
MODEL_SPECS = {
    "Black":  {"label": "Black-Scholes", "price_col": "price_black"},
    "Heston": {"label": "Heston",        "price_col": "price_heston"},
    "SVCJ":   {"label": "SVCJ",          "price_col": "price_svcj"},
}

COLORS = {
    "Black":  "#636EFA",   # Plotly default blue
    "Heston": "#EF553B",   # Plotly default red
    "SVCJ":   "#00CC96",   # Plotly default green
}

FIGDIMS = dict(width=1200, height=1100)


In [2]:

# --- load data ---
from pathlib import Path

# Resolve the Excel path robustly (works both locally and in this sandbox)
p = Path(DATA_PATH)
if not p.exists():
    candidates = [
        Path.cwd() / "calibration_results.xlsx",
        Path("/mnt/data/calibration_results.xlsx"),
        Path.home() / "calibration_results.xlsx",
    ]
    for c in candidates:
        if c.exists():
            p = c
            break

assert p.exists(), f"File not found: {DATA_PATH}. Put 'calibration_results.xlsx' next to this notebook or update DATA_PATH."

black_params = pd.read_excel(p, sheet_name="black_params")
heston_params = pd.read_excel(p, sheet_name="heston_params")
svcj_params = pd.read_excel(p, sheet_name="svcj_params")

train_data = pd.read_excel(p, sheet_name="train_data")
test_data  = pd.read_excel(p, sheet_name="test_data")

# Parse timestamps
for df in (black_params, heston_params, svcj_params):
    df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True)

for df in (train_data, test_data):
    df["snapshot_ts"] = pd.to_datetime(df["snapshot_ts"], utc=True)
    df["expiry_datetime"] = pd.to_datetime(df["expiry_datetime"], utc=True)

print("Loaded from:", p)
print(" - black_params:", black_params.shape)
print(" - heston_params:", heston_params.shape)
print(" - svcj_params:", svcj_params.shape)
print(" - train_data:", train_data.shape)
print(" - test_data :", test_data.shape)

display(black_params.head(3))


Loaded from: calibration_results.xlsx
 - black_params: (294, 16)
 - heston_params: (294, 20)
 - svcj_params: (294, 25)
 - train_data: (86786, 34)
 - test_data : (37391, 34)


,timestamp,currency,success,message,nfev,rmse_fit,mae_fit,rmse_train,mae_train,rmse_test,mae_test,n_options_total,n_train,n_test,random_seed,sigma
0,2025-12-30 17:31:15+00:00,BTC,True,`ftol` termination condition is satisfied.,5,0.005825,0.003963,0.005825,0.003963,0.004958,0.003629,398,278,120,123,0.433832
1,2025-12-30 21:17:51+00:00,BTC,True,`ftol` termination condition is satisfied.,4,0.005975,0.004278,0.005975,0.004278,0.006321,0.003966,392,274,118,124,0.431057
2,2025-12-31 09:18:28+00:00,BTC,True,`ftol` termination condition is satisfied.,5,0.005681,0.003941,0.005681,0.003941,0.004456,0.003473,391,273,118,125,0.445090


## 1) Build snapshot-level “results” tables (metrics + convergence + parameters)

We consolidate the three model-specific parameter sheets into a common long format:

- One row per *(snapshot, currency, model)*  
- With: train/test RMSE & MAE, success flag, solver message, `nfev`, and calibrated parameters.


In [3]:

def _to_long(df, model_name, param_cols):
    base_cols = [
        "timestamp","currency","success","message","nfev",
        "rmse_fit","mae_fit","rmse_train","mae_train","rmse_test","mae_test",
        "n_options_total","n_train","n_test","random_seed"
    ]
    keep = base_cols + param_cols
    out = df[keep].copy()
    out["model"] = model_name
    return out

results_long = pd.concat([
    _to_long(black_params, "Black",  ["sigma"]),
    _to_long(heston_params,"Heston", ["kappa","theta","sigma_v","rho","v0"]),
    _to_long(svcj_params,  "SVCJ",   ["kappa","theta","sigma_v","rho","v0","lam","ell_y","sigma_y","ell_v","rho_j"]),
], ignore_index=True)

results_long = results_long.sort_values(["currency","timestamp","model"]).reset_index(drop=True)

# Convenience: only successful rows (for model-specific parameter analysis)
results_ok = results_long[results_long["success"] == True].copy()

display(results_long.head(6))
print("Currencies:", results_long["currency"].unique())


,timestamp,currency,success,message,nfev,rmse_fit,mae_fit,rmse_train,mae_train,rmse_test,mae_test,n_options_total,n_train,n_test,random_seed,sigma,model,kappa,theta,sigma_v,rho,v0,lam,ell_y,sigma_y,ell_v,rho_j
0,2025-12-30 17:31:15+00:00,BTC,True,`ftol` termination condition is satisfied.,5,0.005825,0.003963,0.005825,0.003963,0.004958,0.003629,398,278,120,123,0.433832,Black,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-12-30 17:31:15+00:00,BTC,True,`ftol` termination condition is satisfied.,30,0.002431,0.001201,0.002431,0.001201,0.001521,0.001009,398,278,120,123,NaN,Heston,5.731859,0.267391,1.755159,-0.214404,0.146301,NaN,NaN,NaN,NaN,NaN
2,2025-12-30 17:31:15+00:00,BTC,True,`ftol` termination condition is satisfied.,55,0.002106,0.000700,0.002106,0.000700,0.001286,0.000503,398,278,120,123,NaN,SVCJ,3.439001,0.095674,0.514598,-0.202940,0.118918,1.184905,-0.064699,0.204991,0.407453,-0.073968
3,2025-12-30 21:17:51+00:00,BTC,True,`ftol` termination condition is satisfied.,4,0.005975,0.004278,0.005975,0.004278,0.006321,0.003966,392,274,118,124,0.431057,Black,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-12-30 21:17:51+00:00,BTC,True,`ftol` termination condition is satisfied.,15,0.002594,0.001377,0.002594,0.001377,0.003193,0.001262,392,274,118,124,NaN,Heston,6.253381,0.263011,1.817491,-0.197900,0.142405,NaN,NaN,NaN,NaN,NaN
5,2025-12-30 21:17:51+00:00,BTC,True,`ftol` termination condition is satisfied.,57,0.002395,0.000868,0.002395,0.000868,0.002778,0.000770,392,274,118,124,NaN,SVCJ,4.219714,0.073935,0.075883,-0.956331,0.112338,1.535474,-0.054393,0.182472,0.413736,-0.066460


Currencies: ['BTC' 'ETH']


## 2) Option-level metrics derived from `train_data` and `test_data`

The parameter sheets already contain RMSE/MAE train/test, but option-level data lets us compute
additional diagnostics (spread-relative errors, within-spread fractions, bucket analyses, etc.).


In [4]:

EPS = 1e-12

def compute_snapshot_metrics_from_quotes(df_quotes: pd.DataFrame, split_name: str) -> pd.DataFrame:
    """Compute per-(currency, snapshot_ts, model) metrics from option-level quotes."""
    out_all = []
    base_cols = ["currency","snapshot_ts","mid_price_clean","bid_ask_spread","spread","log_moneyness","time_to_maturity"]

    # choose spread column: prefer explicit bid_ask_spread, fallback to spread
    spread_col = "bid_ask_spread" if "bid_ask_spread" in df_quotes.columns else "spread"

    for model_key, spec in MODEL_SPECS.items():
        price_col = spec["price_col"]
        if price_col not in df_quotes.columns:
            continue

        tmp = df_quotes[["currency","snapshot_ts","mid_price_clean", spread_col, "log_moneyness","time_to_maturity", price_col]].copy()
        tmp = tmp.rename(columns={price_col:"price_model", spread_col:"spread_abs"})

        # clean
        tmp["mid_price_clean"] = pd.to_numeric(tmp["mid_price_clean"], errors="coerce")
        tmp["price_model"] = pd.to_numeric(tmp["price_model"], errors="coerce")
        tmp["spread_abs"] = pd.to_numeric(tmp["spread_abs"], errors="coerce")

        tmp = tmp[np.isfinite(tmp["mid_price_clean"]) & np.isfinite(tmp["price_model"]) & np.isfinite(tmp["spread_abs"])].copy()
        tmp = tmp[tmp["spread_abs"] > 0].copy()

        err = tmp["price_model"] - tmp["mid_price_clean"]
        abs_err = err.abs()

        tmp["err2"] = err * err
        tmp["abs_err"] = abs_err
        tmp["within_spread"] = (abs_err <= tmp["spread_abs"]).astype(float)
        tmp["within_half_spread"] = (abs_err <= 0.5 * tmp["spread_abs"]).astype(float)
        tmp["abs_err_over_spread"] = abs_err / (tmp["spread_abs"] + EPS)
        tmp["smape"] = 2.0 * abs_err / (tmp["price_model"].abs() + tmp["mid_price_clean"].abs() + EPS)

        g = tmp.groupby(["currency","snapshot_ts"], as_index=False)
        agg = g.agg(
            n=("abs_err","count"),
            mse=("err2","mean"),
            mae=("abs_err","mean"),
            within_spread=("within_spread","mean"),
            within_half_spread=("within_half_spread","mean"),
            abs_err_over_spread=("abs_err_over_spread","mean"),
            smape=("smape","mean"),
            mean_price=("mid_price_clean","mean"),
        )
        agg["rmse"] = np.sqrt(agg["mse"])
        agg["rmse_over_mean_price"] = agg["rmse"] / (agg["mean_price"].abs() + EPS)

        agg["model"] = model_key
        agg["split"] = split_name
        out_all.append(agg)

    out = pd.concat(out_all, ignore_index=True) if out_all else pd.DataFrame()
    return out

opt_metrics_train = compute_snapshot_metrics_from_quotes(train_data, "train")
opt_metrics_test  = compute_snapshot_metrics_from_quotes(test_data,  "test")

opt_metrics = pd.concat([opt_metrics_train, opt_metrics_test], ignore_index=True)
opt_metrics = opt_metrics.sort_values(["currency","snapshot_ts","model","split"]).reset_index(drop=True)

display(opt_metrics.head(6))
print("Option-derived snapshot metrics:", opt_metrics.shape)


,currency,snapshot_ts,n,mse,mae,within_spread,within_half_spread,abs_err_over_spread,smape,mean_price,rmse,rmse_over_mean_price,model,split
0,BTC,2025-12-30 17:31:15+00:00,120,0.000025,0.003629,0.225000,0.150000,3.140859,0.248498,0.083687,0.004958,0.059246,Black,test
1,BTC,2025-12-30 17:31:15+00:00,278,0.000034,0.003963,0.291367,0.233813,2.927041,0.215789,0.121861,0.005825,0.047797,Black,train
2,BTC,2025-12-30 17:31:15+00:00,120,0.000002,0.001009,0.658333,0.408333,1.157901,0.151182,0.083687,0.001521,0.018175,Heston,test
3,BTC,2025-12-30 17:31:15+00:00,278,0.000006,0.001201,0.744604,0.500000,0.887746,0.111333,0.121861,0.002431,0.019947,Heston,train
4,BTC,2025-12-30 17:31:15+00:00,120,0.000002,0.000503,0.925000,0.808333,0.329393,0.028150,0.083687,0.001286,0.015363,SVCJ,test
5,BTC,2025-12-30 17:31:15+00:00,278,0.000004,0.000700,0.960432,0.874101,0.247692,0.019607,0.121861,0.002106,0.017280,SVCJ,train


Option-derived snapshot metrics: (1742, 14)


## 3) Bootstrap helpers (snapshot-level)

We treat each snapshot as one observation. For a snapshot-level series \(m_t\), we report:
- mean + 95% bootstrap CI for the mean (percentile bootstrap),
- median, quartiles, standard deviation, and sample size.


In [5]:

def bootstrap_mean_ci(values: np.ndarray, n_boot: int = 3000, alpha: float = 0.05, rng: np.random.Generator = RNG):
    values = np.asarray(values, dtype=float)
    values = values[np.isfinite(values)]
    n = len(values)
    if n == 0:
        return np.nan, np.nan, np.nan
    idx = rng.integers(0, n, size=(n_boot, n))
    boot_means = values[idx].mean(axis=1)
    lo = np.quantile(boot_means, alpha/2)
    hi = np.quantile(boot_means, 1 - alpha/2)
    return values.mean(), lo, hi

def summarize_snapshot_series(values: pd.Series, n_boot: int = 3000) -> dict:
    arr = pd.to_numeric(values, errors="coerce").to_numpy(dtype=float)
    arr = arr[np.isfinite(arr)]
    if len(arr) == 0:
        return dict(n=0, mean=np.nan, ci_low=np.nan, ci_high=np.nan,
                    median=np.nan, q25=np.nan, q75=np.nan, std=np.nan, min=np.nan, max=np.nan)
    mean, lo, hi = bootstrap_mean_ci(arr, n_boot=n_boot)
    return dict(
        n=len(arr),
        mean=float(mean), ci_low=float(lo), ci_high=float(hi),
        median=float(np.median(arr)),
        q25=float(np.quantile(arr, 0.25)),
        q75=float(np.quantile(arr, 0.75)),
        std=float(np.std(arr, ddof=1)) if len(arr) > 1 else 0.0,
        min=float(np.min(arr)),
        max=float(np.max(arr)),
    )


## 4) Plot helpers (time-series and boxplots)

We keep the same **2×2 subplot layout** you already use:

1) RMSE (all models)  
2) MAE (all models)  
3) RMSE (Heston vs SVCJ)  
4) MAE (Heston vs SVCJ)


In [6]:
def add_line(fig, row, col, df, xcol, ycol, name, color):
    # If repeated timestamps exist, aggregate by mean (mirrors your earlier behavior)
    s = df.groupby(xcol, as_index=False)[ycol].mean()
    fig.add_trace(
        go.Scatter(x=s[xcol], y=s[ycol], mode="lines", line=dict(color=color, width=2), name=name, showlegend=False),
        row=row, col=col
    )

def add_box(fig, row, col, values, name, color):
    fig.add_trace(
        go.Box(
            y=values,
            name=name,
            marker_color=color,
            boxmean=True,
            boxpoints="outliers",
            jitter=0.15,
            pointpos=0.0,
            showlegend=False,
        ),
        row=row, col=col
    )

def _subplot_axis_suffix_2x2(row: int, col: int) -> str:
    # 2x2 numbering: (1,1)->"", (1,2)->"2", (2,1)->"3", (2,2)->"4"
    idx = (row - 1) * 2 + col
    return "" if idx == 1 else str(idx)

def add_subplot_legend(fig, row: int, col: int, model_keys: list, font_size: int = 12):
    """Emulate 'one legend per subplot' using an annotation box in the subplot domain."""
    suf = _subplot_axis_suffix_2x2(row, col)
    xref = f"x{suf} domain" if suf else "x domain"
    yref = f"y{suf} domain" if suf else "y domain"

    lines = []
    for mk in model_keys:
        label = MODEL_SPECS[mk]["label"]
        color = COLORS[mk]
        lines.append(f"<span style='color:{color}'>●</span> {label}")
    text = "<br>".join(lines)

    fig.add_annotation(
        x=0.01, y=0.99,
        xref=xref, yref=yref,
        text=text,
        showarrow=False,
        align="left",
        bgcolor="rgba(255,255,255,0.70)",
        bordercolor="rgba(0,0,0,0.15)",
        borderwidth=1,
        font=dict(size=font_size),
    )

def plot_error_timeseries(results_long_df: pd.DataFrame, currency: str, split: str = "test"):
    metric_rmse = f"rmse_{split}"
    metric_mae  = f"mae_{split}"
    df = results_long_df[(results_long_df["currency"] == currency) & (results_long_df["success"] == True)].copy()
    df = df.sort_values("timestamp")

    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            f"RMSE {split.title()}",
            f"MAE {split.title()}",
            f"RMSE {split.title()} – Heston vs SVCJ",
            f"MAE {split.title()} – Heston vs SVCJ",
        ],
        vertical_spacing=0.08,
        horizontal_spacing=0.08,
    )

    for model in ["Black","Heston","SVCJ"]:
        sub = df[df["model"] == model]
        add_line(fig, 1, 1, sub, "timestamp", metric_rmse, MODEL_SPECS[model]["label"], COLORS[model])
        add_line(fig, 1, 2, sub, "timestamp", metric_mae,  MODEL_SPECS[model]["label"], COLORS[model])

    for model in ["Heston","SVCJ"]:
        sub = df[df["model"] == model]
        add_line(fig, 2, 1, sub, "timestamp", metric_rmse, MODEL_SPECS[model]["label"], COLORS[model])
        add_line(fig, 2, 2, sub, "timestamp", metric_mae,  MODEL_SPECS[model]["label"], COLORS[model])

    # One legend box per subplot (annotation-based)
    add_subplot_legend(fig, 1, 1, ["Black","Heston","SVCJ"])
    add_subplot_legend(fig, 1, 2, ["Black","Heston","SVCJ"])
    add_subplot_legend(fig, 2, 1, ["Heston","SVCJ"])
    add_subplot_legend(fig, 2, 2, ["Heston","SVCJ"])

    fig.update_layout(
        title=f"{currency} — {split.title()} error time series (snapshot-level)",
        showlegend=False,
        **FIGDIMS
    )
    fig.update_yaxes(title_text="RMSE (coin premium)", row=1, col=1)
    fig.update_yaxes(title_text="MAE (coin premium)",  row=1, col=2)
    fig.update_yaxes(title_text="RMSE (coin premium)", row=2, col=1)
    fig.update_yaxes(title_text="MAE (coin premium)",  row=2, col=2)
    return fig

def plot_error_boxplots(results_long_df: pd.DataFrame, currency: str, split: str = "test"):
    metric_rmse = f"rmse_{split}"
    metric_mae  = f"mae_{split}"
    df = results_long_df[(results_long_df["currency"] == currency) & (results_long_df["success"] == True)].copy()

    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            f"RMSE {split.title()} (distribution across snapshots)",
            f"MAE {split.title()} (distribution across snapshots)",
            f"RMSE {split.title()} – Heston vs SVCJ",
            f"MAE {split.title()} – Heston vs SVCJ",
        ],
        vertical_spacing=0.12,
        horizontal_spacing=0.12,
    )

    for model in ["Black","Heston","SVCJ"]:
        vals_rmse = df.loc[df["model"] == model, metric_rmse].dropna().values
        vals_mae  = df.loc[df["model"] == model, metric_mae].dropna().values
        add_box(fig, 1, 1, vals_rmse, MODEL_SPECS[model]["label"], COLORS[model])
        add_box(fig, 1, 2, vals_mae,  MODEL_SPECS[model]["label"], COLORS[model])

    for model in ["Heston","SVCJ"]:
        vals_rmse = df.loc[df["model"] == model, metric_rmse].dropna().values
        vals_mae  = df.loc[df["model"] == model, metric_mae].dropna().values
        add_box(fig, 2, 1, vals_rmse, MODEL_SPECS[model]["label"], COLORS[model])
        add_box(fig, 2, 2, vals_mae,  MODEL_SPECS[model]["label"], COLORS[model])

    add_subplot_legend(fig, 1, 1, ["Black","Heston","SVCJ"])
    add_subplot_legend(fig, 1, 2, ["Black","Heston","SVCJ"])
    add_subplot_legend(fig, 2, 1, ["Heston","SVCJ"])
    add_subplot_legend(fig, 2, 2, ["Heston","SVCJ"])

    fig.update_layout(
        title=f"{currency} — {split.title()} error boxplots (snapshot-level)",
        showlegend=False,
        **FIGDIMS
    )
    fig.update_yaxes(title_text="RMSE (coin premium)", row=1, col=1)
    fig.update_yaxes(title_text="MAE (coin premium)",  row=1, col=2)
    fig.update_yaxes(title_text="RMSE (coin premium)", row=2, col=1)
    fig.update_yaxes(title_text="MAE (coin premium)",  row=2, col=2)
    return fig


## 5) Summary tables (errors + CI bands + incremental gains)

This produces:
- per-model summary stats (mean + 95% CI, median, quartiles, etc.)
- incremental gains and win rates for:
  - Heston vs Black
  - SVCJ vs Heston


In [7]:

def error_summary_table(results_long_df: pd.DataFrame, currency: str, split: str = "test", n_boot: int = 3000) -> pd.DataFrame:
    metric_cols = [(f"rmse_{split}", "RMSE"), (f"mae_{split}", "MAE")]

    df = results_long_df[(results_long_df["currency"] == currency) & (results_long_df["success"] == True)].copy()
    df = df.sort_values("timestamp")

    rows = []
    for col, metric_name in metric_cols:
        for model in ["Black","Heston","SVCJ"]:
            vals = df.loc[df["model"] == model, col]
            s = summarize_snapshot_series(vals, n_boot=n_boot)
            rows.append({
                "currency": currency, "split": split, "metric": metric_name, "item": model,
                **s
            })

        # incremental gains
        wide = df.pivot_table(index="timestamp", columns="model", values=col, aggfunc="mean")
        if {"Black","Heston","SVCJ"}.issubset(wide.columns):
            # Heston vs Black
            diff_hb = wide["Black"] - wide["Heston"]
            pct_hb  = diff_hb / wide["Black"]
            s_diff = summarize_snapshot_series(diff_hb, n_boot=n_boot)
            s_pct  = summarize_snapshot_series(pct_hb,  n_boot=n_boot)
            win = float((wide["Heston"] < wide["Black"]).mean())
            rows.append({"currency": currency, "split": split, "metric": metric_name, "item": "GAIN: Black→Heston (abs)", **s_diff, "win_rate": win})
            rows.append({"currency": currency, "split": split, "metric": metric_name, "item": "GAIN: Black→Heston (%)",   **s_pct,  "win_rate": win})

            # SVCJ vs Heston
            diff_sh = wide["Heston"] - wide["SVCJ"]
            pct_sh  = diff_sh / wide["Heston"]
            s_diff = summarize_snapshot_series(diff_sh, n_boot=n_boot)
            s_pct  = summarize_snapshot_series(pct_sh,  n_boot=n_boot)
            win = float((wide["SVCJ"] < wide["Heston"]).mean())
            rows.append({"currency": currency, "split": split, "metric": metric_name, "item": "GAIN: Heston→SVCJ (abs)", **s_diff, "win_rate": win})
            rows.append({"currency": currency, "split": split, "metric": metric_name, "item": "GAIN: Heston→SVCJ (%)",   **s_pct,  "win_rate": win})

    out = pd.DataFrame(rows)

    # format CI as a string too
    out["ci_95"] = out.apply(lambda r: f"[{r['ci_low']:.6g}, {r['ci_high']:.6g}]" if pd.notna(r["ci_low"]) else "", axis=1)

    # reorder
    cols = ["currency","split","metric","item","n","mean","ci_95","median","q25","q75","std","min","max","win_rate"]
    for c in cols:
        if c not in out.columns:
            out[c] = np.nan
    return out[cols].sort_values(["metric","item"]).reset_index(drop=True)

# Example call (uncomment to view quickly)
# display(error_summary_table(results_long, "BTC", split="test"))


## 6) Convergence diagnostics (success, termination messages, nfev)

We summarize by *(currency, model)*:
- number of snapshots
- success rate
- `nfev` distribution (median / p90 / max)
- how often the solver hit the max evaluation cap (detected from termination message)


In [8]:

def convergence_table(results_long_df: pd.DataFrame) -> pd.DataFrame:
    df = results_long_df.copy()
    df["hit_cap"] = df["message"].astype(str).str.contains("maximum number of function evaluations", case=False, regex=False)
    g = df.groupby(["currency","model"], as_index=False)

    out = g.agg(
        n_snapshots=("timestamp","count"),
        success_rate=("success","mean"),
        nfev_median=("nfev","median"),
        nfev_mean=("nfev","mean"),
        nfev_p90=("nfev", lambda x: float(np.quantile(pd.to_numeric(x, errors="coerce"), 0.90))),
        nfev_max=("nfev","max"),
        hit_cap_rate=("hit_cap","mean"),
    )

    # top messages
    msg = (df.groupby(["currency","model","message"])
             .size()
             .reset_index(name="count")
             .sort_values(["currency","model","count"], ascending=[True,True,False]))
    top_msgs = msg.groupby(["currency","model"]).head(3)
    top_msgs["rank"] = top_msgs.groupby(["currency","model"]).cumcount()+1
    top_msgs = top_msgs.pivot_table(index=["currency","model"], columns="rank", values="message", aggfunc="first").reset_index()
    top_msgs = top_msgs.rename(columns={1:"top_message_1",2:"top_message_2",3:"top_message_3"})

    out = out.merge(top_msgs, on=["currency","model"], how="left")
    return out.sort_values(["currency","model"]).reset_index(drop=True)

display(convergence_table(results_long))


/var/folders/nc/_vy7mzns49vfq0r0kq6c35km0000gn/T/ipykernel_71677/761420318.py:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,currency,model,n_snapshots,success_rate,nfev_median,nfev_mean,nfev_p90,nfev_max,hit_cap_rate,top_message_1,top_message_2,top_message_3
0,BTC,Black,147,1.00000,4.0,4.353741,5.0,6,0.00000,`ftol` termination condition is satisfied.,NaN,NaN
1,BTC,Heston,147,1.00000,10.0,12.217687,20.0,57,0.00000,`ftol` termination condition is satisfied.,NaN,NaN
2,BTC,SVCJ,147,1.00000,20.0,25.136054,37.4,158,0.00000,`xtol` termination condition is satisfied.,`ftol` termination condition is satisfied.,Both `ftol` and `xtol` termination conditions ...
3,ETH,Black,147,1.00000,4.0,4.217687,5.0,6,0.00000,`ftol` termination condition is satisfied.,NaN,NaN
4,ETH,Heston,147,1.00000,8.0,9.360544,12.4,44,0.00000,`ftol` termination condition is satisfied.,NaN,NaN
5,ETH,SVCJ,147,0.92517,29.0,51.163265,118.8,200,0.07483,`ftol` termination condition is satisfied.,`xtol` termination condition is satisfied.,The maximum number of function evaluations is ...


## 7) Spread-relative diagnostics (test and train)

Using option-level quotes, we compute per-snapshot:
- fraction of quotes priced within **half-spread** and within the **full spread**
- mean \(|error|/spread\)
- sMAPE and RMSE normalized by mean market premium

We plot these over time and summarize them with bootstrap CIs.


In [9]:
def spread_metric_timeseries(opt_metrics_df: pd.DataFrame, currency: str, split: str = "test"):
    df = opt_metrics_df[(opt_metrics_df["currency"] == currency) & (opt_metrics_df["split"] == split)].copy()
    df = df.sort_values("snapshot_ts")

    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            "Within spread (fraction)",
            "Within half-spread (fraction)",
            "Mean |error| / spread",
            "sMAPE",
        ],
        vertical_spacing=0.10,
        horizontal_spacing=0.10,
    )

    for model in ["Black","Heston","SVCJ"]:
        sub = df[df["model"] == model]
        add_line(fig, 1, 1, sub, "snapshot_ts", "within_spread", MODEL_SPECS[model]["label"], COLORS[model])
        add_line(fig, 1, 2, sub, "snapshot_ts", "within_half_spread", MODEL_SPECS[model]["label"], COLORS[model])
        add_line(fig, 2, 1, sub, "snapshot_ts", "abs_err_over_spread", MODEL_SPECS[model]["label"], COLORS[model])
        add_line(fig, 2, 2, sub, "snapshot_ts", "smape", MODEL_SPECS[model]["label"], COLORS[model])

    # One legend box per subplot (annotation-based)
    add_subplot_legend(fig, 1, 1, ["Black","Heston","SVCJ"])
    add_subplot_legend(fig, 1, 2, ["Black","Heston","SVCJ"])
    add_subplot_legend(fig, 2, 1, ["Black","Heston","SVCJ"])
    add_subplot_legend(fig, 2, 2, ["Black","Heston","SVCJ"])

    fig.update_layout(
        title=f"{currency} — {split.title()} spread-relative diagnostics (per snapshot)",
        showlegend=False,
        width=1200, height=900
    )
    return fig

def spread_metric_summary_table(opt_metrics_df: pd.DataFrame, currency: str, split: str = "test", n_boot: int = 3000) -> pd.DataFrame:
    df = opt_metrics_df[(opt_metrics_df["currency"] == currency) & (opt_metrics_df["split"] == split)].copy()
    rows=[]
    for model in ["Black","Heston","SVCJ"]:
        sub = df[df["model"] == model]
        for col, metric in [
            ("within_spread","within_spread"),
            ("within_half_spread","within_half_spread"),
            ("abs_err_over_spread","abs_err_over_spread"),
            ("smape","sMAPE"),
            ("rmse_over_mean_price","rmse_over_mean_price"),
        ]:
            s = summarize_snapshot_series(sub[col], n_boot=n_boot)
            rows.append({"currency":currency,"split":split,"model":model,"metric":metric, **s, "ci_95": f"[{s['ci_low']:.6g}, {s['ci_high']:.6g}]"})
    out=pd.DataFrame(rows)
    return out[["currency","split","model","metric","n","mean","ci_95","median","q25","q75","std","min","max"]].sort_values(["metric","model"])

# Example: display summary for BTC test
# display(spread_metric_summary_table(opt_metrics, "BTC", split="test"))


## 8) Error breakdowns by moneyness and maturity (test set)

We report **MAE** broken down by:
- absolute log-moneyness \(|\log(K/F)|\) buckets
- maturity buckets (based on time-to-maturity in years)

Bucket metrics are computed **within each snapshot**, then averaged across snapshots (equal-weighted over time).


In [10]:

# Bucket edges
MONEY_BINS = [0.0, 0.05, 0.15, 0.30, np.inf]
MONEY_LABELS = ["|log(K/F)|≤0.05", "0.05–0.15", "0.15–0.30", ">0.30"]

# Time to maturity in years: 1w, 1m, 3m
T_BINS = [0.0, 7/365, 30/365, 90/365, np.inf]
T_LABELS = ["≤1w", "1w–1m", "1m–3m", ">3m"]

def _add_buckets(df):
    out = df.copy()
    out["abs_log_moneyness"] = out["log_moneyness"].abs()
    out["m_bucket"] = pd.cut(out["abs_log_moneyness"], bins=MONEY_BINS, labels=MONEY_LABELS, right=True, include_lowest=True)
    out["t_bucket"] = pd.cut(out["time_to_maturity"], bins=T_BINS, labels=T_LABELS, right=True, include_lowest=True)
    return out

def bucket_mae_by_snapshot(df_quotes: pd.DataFrame, currency: str, split: str = "test"):
    df = df_quotes[df_quotes["currency"] == currency].copy()
    df = _add_buckets(df)

    res = []
    for model_key, spec in MODEL_SPECS.items():
        price_col = spec["price_col"]
        if price_col not in df.columns:
            continue
        tmp = df[["snapshot_ts","m_bucket","t_bucket","mid_price_clean", price_col]].copy()
        tmp = tmp.rename(columns={price_col:"price_model"})
        tmp["mid_price_clean"] = pd.to_numeric(tmp["mid_price_clean"], errors="coerce")
        tmp["price_model"] = pd.to_numeric(tmp["price_model"], errors="coerce")
        tmp = tmp[np.isfinite(tmp["mid_price_clean"]) & np.isfinite(tmp["price_model"])].copy()
        tmp["abs_err"] = (tmp["price_model"] - tmp["mid_price_clean"]).abs()

        # moneyness bucket MAE per snapshot
        g1 = tmp.groupby(["snapshot_ts","m_bucket"], as_index=False)["abs_err"].mean()
        g1["model"] = model_key
        g1["split"] = split
        g1["bucket_type"] = "moneyness"
        g1 = g1.rename(columns={"m_bucket":"bucket","abs_err":"mae"})
        res.append(g1)

        # maturity bucket MAE per snapshot
        g2 = tmp.groupby(["snapshot_ts","t_bucket"], as_index=False)["abs_err"].mean()
        g2["model"] = model_key
        g2["split"] = split
        g2["bucket_type"] = "maturity"
        g2 = g2.rename(columns={"t_bucket":"bucket","abs_err":"mae"})
        res.append(g2)

    out = pd.concat(res, ignore_index=True)
    out["currency"] = currency
    return out

bucket_btc = bucket_mae_by_snapshot(test_data, "BTC", split="test")
bucket_eth = bucket_mae_by_snapshot(test_data, "ETH", split="test")
bucket_all = pd.concat([bucket_btc, bucket_eth], ignore_index=True)

display(bucket_all.head(6))


/var/folders/nc/_vy7mzns49vfq0r0kq6c35km0000gn/T/ipykernel_71677/472583268.py:33: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/var/folders/nc/_vy7mzns49vfq0r0kq6c35km0000gn/T/ipykernel_71677/472583268.py:41: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/var/folders/nc/_vy7mzns49vfq0r0kq6c35km0000gn/T/ipykernel_71677/472583268.py:33: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/var/folders/nc/_vy7mzns49vfq0r0kq6c35km

,snapshot_ts,bucket,mae,model,split,bucket_type,currency
0,2025-12-30 17:31:15+00:00,|log(K/F)|≤0.05,0.003093,Black,test,moneyness,BTC
1,2025-12-30 17:31:15+00:00,0.05–0.15,0.003357,Black,test,moneyness,BTC
2,2025-12-30 17:31:15+00:00,0.15–0.30,0.004060,Black,test,moneyness,BTC
3,2025-12-30 17:31:15+00:00,>0.30,0.004229,Black,test,moneyness,BTC
4,2025-12-30 21:17:51+00:00,|log(K/F)|≤0.05,0.003925,Black,test,moneyness,BTC
5,2025-12-30 21:17:51+00:00,0.05–0.15,0.003334,Black,test,moneyness,BTC


In [11]:

def bucket_summary_table(bucket_df: pd.DataFrame, currency: str, bucket_type: str, n_boot: int = 2000) -> pd.DataFrame:
    df = bucket_df[(bucket_df["currency"]==currency) & (bucket_df["bucket_type"]==bucket_type)].copy()
    rows=[]
    for model in ["Black","Heston","SVCJ"]:
        sub = df[df["model"]==model]
        for b in sub["bucket"].dropna().unique():
            vals = sub[sub["bucket"]==b].groupby("snapshot_ts")["mae"].mean()  # ensure one value per snapshot
            s = summarize_snapshot_series(vals, n_boot=n_boot)
            rows.append({"currency":currency,"bucket_type":bucket_type,"bucket":str(b),"model":model, **s,
                         "ci_95": f"[{s['ci_low']:.6g}, {s['ci_high']:.6g}]"})
    out=pd.DataFrame(rows)
    return out[["currency","bucket_type","bucket","model","n","mean","ci_95","median","q25","q75"]].sort_values(["bucket","model"])

def plot_bucket_bars(bucket_df: pd.DataFrame, currency: str, bucket_type: str):
    df = bucket_df[(bucket_df["currency"]==currency) & (bucket_df["bucket_type"]==bucket_type)].copy()
    # average across snapshots (equal-weight)
    df_mean = df.groupby(["model","bucket"], as_index=False)["mae"].mean()
    order = MONEY_LABELS if bucket_type=="moneyness" else T_LABELS
    df_mean["bucket"] = pd.Categorical(df_mean["bucket"], categories=order, ordered=True)
    df_mean = df_mean.sort_values("bucket")

    fig = go.Figure()
    for model in ["Black","Heston","SVCJ"]:
        sub = df_mean[df_mean["model"]==model]
        fig.add_trace(go.Bar(
            x=sub["bucket"].astype(str),
            y=sub["mae"],
            name=MODEL_SPECS[model]["label"],
            marker_color=COLORS[model],
        ))
    fig.update_layout(
        title=f"{currency} — Test MAE by {bucket_type} bucket (snapshot-equal-weighted)",
        barmode="group",
        width=1100, height=500
    )
    fig.update_yaxes(title_text="MAE (coin premium)")
    return fig

# Example quick views (uncomment if desired)
# display(bucket_summary_table(bucket_all, "BTC", "moneyness"))
# plot_bucket_bars(bucket_all, "BTC", "moneyness").show()


## 9) Parameter stability and bound-pressure diagnostics

We provide:
- time-series plots for key parameters (Heston and SVCJ),
- distribution boxplots,
- “near-bound” rates using the calibration bounds (in natural parameter space),
- and the Heston/SVCJ **Feller ratio** \(\sigma_v^2/(2\kappa\theta)\) as a constraint-pressure proxy.


In [12]:

# Natural-space bounds implied by calibration packing/unpacking (see src/calibration.py)
RHO_LB = np.tanh(-5.0)
RHO_UB = np.tanh( 5.0)

BOUNDS = {
    "Black": {"sigma": (1e-4, 5.0)},
    "Heston": {
        "kappa": (1e-4, 50.0),
        "theta": (1e-6, 5.0),
        "sigma_v": (1e-4, 10.0),
        "rho": (RHO_LB, RHO_UB),
        "v0": (1e-6, 5.0),
    },
    "SVCJ": {
        "kappa": (1e-4, 50.0),
        "theta": (1e-6, 5.0),
        "sigma_v": (1e-4, 10.0),
        "rho": (RHO_LB, RHO_UB),
        "v0": (1e-6, 5.0),
        "lam": (1e-6, 10.0),
        "ell_y": (-5.0, 5.0),
        "sigma_y": (1e-4, 5.0),
        "ell_v": (1e-6, 10.0),
        "rho_j": (RHO_LB, RHO_UB),
    },
}

def add_feller_ratio(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    if {"kappa","theta","sigma_v"}.issubset(out.columns):
        out["feller_ratio"] = (out["sigma_v"]**2) / (2.0*out["kappa"]*out["theta"] + EPS)
    return out

results_ok = add_feller_ratio(results_ok)

def near_bound_rates(df: pd.DataFrame, model: str, tol: float = 0.02) -> pd.DataFrame:
    """Share of calibrations within tol*(ub-lb) of lower/upper bound."""
    bounds = BOUNDS[model]
    sub = df[(df["model"]==model) & (df["success"]==True)].copy()
    out=[]
    for p,(lb,ub) in bounds.items():
        if p not in sub.columns:
            continue
        x = pd.to_numeric(sub[p], errors="coerce").to_numpy(dtype=float)
        x = x[np.isfinite(x)]
        if len(x)==0:
            continue
        rng = (ub - lb)
        lo = np.mean((x - lb) <= tol*rng)
        hi = np.mean((ub - x) <= tol*rng)
        out.append({"model":model,"param":p,"lb":lb,"ub":ub,"near_lb_rate":lo,"near_ub_rate":hi,
                    "min":float(np.min(x)),"q25":float(np.quantile(x,0.25)),"median":float(np.median(x)),"q75":float(np.quantile(x,0.75)),"max":float(np.max(x))})
    return pd.DataFrame(out).sort_values(["model","param"]).reset_index(drop=True)

# Example (uncomment):
# display(near_bound_rates(results_long[results_long["currency"]=="BTC"], "Heston"))


In [13]:

def plot_param_timeseries(results_long_df: pd.DataFrame, currency: str, model: str, params: list, title: str):
    df = results_long_df[(results_long_df["currency"]==currency) & (results_long_df["model"]==model) & (results_long_df["success"]==True)].copy()
    df = add_feller_ratio(df).sort_values("timestamp")

    n = len(params)
    ncols = 2
    nrows = int(np.ceil(n / ncols))

    fig = make_subplots(
        rows=nrows, cols=ncols,
        subplot_titles=params,
        vertical_spacing=0.10,
        horizontal_spacing=0.10
    )

    for i, p in enumerate(params):
        r = i//ncols + 1
        c = i%ncols + 1
        fig.add_trace(go.Scatter(
            x=df["timestamp"], y=df[p],
            mode="lines",
            line=dict(color=COLORS.get(model, "#333333"), width=2),
            name=p,
            showlegend=False
        ), row=r, col=c)

    fig.update_layout(title=title, width=1200, height=320*nrows)
    return fig

def plot_param_boxplots(results_long_df: pd.DataFrame, currency: str, model: str, params: list, title: str):
    df = results_long_df[(results_long_df["currency"]==currency) & (results_long_df["model"]==model) & (results_long_df["success"]==True)].copy()
    df = add_feller_ratio(df)

    n = len(params)
    ncols = 2
    nrows = int(np.ceil(n / ncols))

    fig = make_subplots(
        rows=nrows, cols=ncols,
        subplot_titles=params,
        vertical_spacing=0.12,
        horizontal_spacing=0.12
    )

    for i, p in enumerate(params):
        r = i//ncols + 1
        c = i%ncols + 1
        add_box(fig, r, c, df[p].dropna().values, p, COLORS.get(model, "#333333"))

    fig.update_layout(title=title, width=1200, height=320*nrows, showlegend=False)
    return fig


## 10) A single “report runner” per currency

To keep the notebook readable, we wrap the repeated steps into one function that:
- prints key counts,
- displays error time series + boxplots (train & test),
- outputs error summary tables (train & test),
- outputs spread-relative diagnostics (train & test),
- outputs bucket plots (test),
- outputs convergence diagnostics (already global),
- outputs parameter stability and near-bound tables.


In [14]:

def run_currency_report(currency: str, n_boot: int = 3000):
    print("="*90)
    print(f"REPORT — {currency}")
    print("="*90)

    # basic coverage
    sub = results_long[results_long["currency"]==currency]
    snap_count = sub["timestamp"].nunique()
    print("Snapshots:", snap_count)
    print("Success rates:")
    display(sub.groupby("model")["success"].mean().to_frame("success_rate"))

    # ---------------- errors (train/test) ----------------
    for split in ["train","test"]:
        fig_ts = plot_error_timeseries(results_long, currency, split=split)
        fig_ts.show()

        fig_box = plot_error_boxplots(results_long, currency, split=split)
        fig_box.show()

        tbl = error_summary_table(results_long, currency, split=split, n_boot=n_boot)
        print(f"Summary table — {currency} / {split}")
        display(tbl)

    # ---------------- spread-relative diagnostics (train/test) ----------------
    for split in ["train","test"]:
        fig_sp = spread_metric_timeseries(opt_metrics, currency, split=split)
        fig_sp.show()

        tbl_sp = spread_metric_summary_table(opt_metrics, currency, split=split, n_boot=n_boot)
        print(f"Spread-relative summary — {currency} / {split}")
        display(tbl_sp)

    # ---------------- bucket analyses (test) ----------------
    print("Bucket tables (test) — moneyness & maturity")
    display(bucket_summary_table(bucket_all, currency, "moneyness", n_boot=max(1000, n_boot//2)))
    display(bucket_summary_table(bucket_all, currency, "maturity",  n_boot=max(1000, n_boot//2)))

    plot_bucket_bars(bucket_all, currency, "moneyness").show()
    plot_bucket_bars(bucket_all, currency, "maturity").show()

    # ---------------- parameter stability ----------------
    print("Parameter stability — Heston")
    hes_params = ["kappa","theta","sigma_v","rho","v0","feller_ratio"]
    plot_param_timeseries(results_long, currency, "Heston", hes_params, f"{currency} — Heston parameter time series").show()
    plot_param_boxplots(results_long, currency, "Heston", hes_params, f"{currency} — Heston parameter distributions").show()
    display(near_bound_rates(results_long[results_long["currency"]==currency], "Heston"))

    print("Parameter stability — SVCJ")
    svcj_params = ["kappa","theta","sigma_v","rho","v0","lam","ell_y","sigma_y","ell_v","rho_j","feller_ratio"]
    plot_param_timeseries(results_long, currency, "SVCJ", svcj_params, f"{currency} — SVCJ parameter time series").show()
    plot_param_boxplots(results_long, currency, "SVCJ", svcj_params, f"{currency} — SVCJ parameter distributions").show()
    display(near_bound_rates(results_long[results_long["currency"]==currency], "SVCJ"))

# ---- run reports (toggle) ----
RUN_REPORTS = True
N_BOOT = 3000  # increase for thesis-grade CIs; lower (e.g. 1000) for faster iteration

if RUN_REPORTS:
    run_currency_report("BTC", n_boot=N_BOOT)
    run_currency_report("ETH", n_boot=N_BOOT)
else:
    print("RUN_REPORTS=False. Set RUN_REPORTS=True to generate the full BTC/ETH report outputs.")


REPORT — BTC
Snapshots: 147
Success rates:


,success_rate
model,
Black,1.0
Heston,1.0
SVCJ,1.0


Summary table — BTC / train


,currency,split,metric,item,n,mean,ci_95,median,q25,q75,std,min,max,win_rate
0,BTC,train,MAE,Black,147,0.003788,"[0.00362722, 0.00397232]",0.003892,0.003106,0.004241,0.001073,0.001887,0.011707,NaN
1,BTC,train,MAE,GAIN: Black→Heston (%),147,0.695227,"[0.666045, 0.722809]",0.737786,0.609432,0.832183,0.176965,0.088809,0.893852,1.000000
2,BTC,train,MAE,GAIN: Black→Heston (abs),147,0.002720,"[0.00254285, 0.00290712]",0.002951,0.001842,0.003435,0.001136,0.000244,0.008569,1.000000
3,BTC,train,MAE,GAIN: Heston→SVCJ (%),147,0.171875,"[0.145346, 0.197225]",0.193626,0.099031,0.280027,0.155023,-0.600083,0.470116,0.877551
4,BTC,train,MAE,GAIN: Heston→SVCJ (abs),147,0.000192,"[0.0001581, 0.000227723]",0.000168,0.000074,0.000285,0.000217,-0.000457,0.000997,0.877551
5,BTC,train,MAE,Heston,147,0.001068,"[0.000986969, 0.00114994]",0.000957,0.000649,0.001278,0.000515,0.000457,0.003138,NaN
6,BTC,train,MAE,SVCJ,147,0.000875,"[0.000808058, 0.000951436]",0.000814,0.000511,0.001066,0.000444,0.000302,0.003339,NaN
7,BTC,train,RMSE,Black,147,0.005685,"[0.00545558, 0.00593341]",0.005759,0.004618,0.006351,0.001514,0.002757,0.016335,NaN
8,BTC,train,RMSE,GAIN: Black→Heston (%),147,0.662356,"[0.624408, 0.699489]",0.754194,0.497316,0.858204,0.225394,-0.009413,0.907382,0.993197
9,BTC,train,RMSE,GAIN: Black→Heston (abs),147,0.003923,"[0.00362903, 0.00420919]",0.003970,0.002294,0.005354,0.001865,-0.000037,0.010782,0.993197


Summary table — BTC / test


...

,currency,bucket_type,bucket,model,n,mean,ci_95,median,q25,q75
2,ETH,maturity,1m–3m,Black,147,0.003162,"[0.00301575, 0.00332951]",0.002988,0.002586,0.003465
6,ETH,maturity,1m–3m,Heston,147,0.001382,"[0.00123911, 0.00154055]",0.001201,0.000658,0.001903
10,ETH,maturity,1m–3m,SVCJ,136,0.001102,"[0.000971554, 0.0012486]",0.000905,0.000489,0.001460
1,ETH,maturity,1w–1m,Black,147,0.003270,"[0.00310562, 0.0034472]",0.003065,0.002623,0.003903
5,ETH,maturity,1w–1m,Heston,147,0.000964,"[0.000865636, 0.00106757]",0.000792,0.000565,0.001126
9,ETH,maturity,1w–1m,SVCJ,136,0.000785,"[0.000685272, 0.000894161]",0.000617,0.000390,0.000916
3,ETH,maturity,>3m,Black,147,0.008786,"[0.00803469, 0.00967983]",0.007865,0.005774,0.010690
7,ETH,maturity,>3m,Heston,147,0.002332,"[0.00208058, 0.002578]",0.002040,0.001104,0.003216
11,ETH,maturity,>3m,SVCJ,136,0.002010,"[0.00174887, 0.00229327]",0.001498,0.000863,0.002627
0,ETH,maturity,≤1w,Black,147,0.002862,"[0.00260985, 0.00310982]",0.002477,0.001585,0.003746


Parameter stability — Heston


,model,param,lb,ub,near_lb_rate,near_ub_rate,min,q25,median,q75,max
0,Heston,kappa,0.000100,50.000000,0.000000,0.136054,5.677636,9.745780,17.661454,35.034284,50.000000
1,Heston,rho,-0.999909,0.999909,0.000000,0.000000,-0.551845,-0.206260,-0.173173,-0.146779,-0.077756
2,Heston,sigma_v,0.000100,10.000000,0.000000,0.000000,2.371161,2.990290,3.997927,5.610481,7.422015
3,Heston,theta,0.000001,5.000000,0.000000,0.000000,0.408451,0.446852,0.456817,0.471654,0.550862
4,Heston,v0,0.000001,5.000000,0.020408,0.000000,0.056826,0.212880,0.265792,0.332972,2.334186


Parameter stability — SVCJ


,model,param,lb,ub,near_lb_rate,near_ub_rate,min,q25,median,q75,max
0,SVCJ,ell_v,0.000001,10.000000,0.125000,0.154412,0.000001,0.422299,0.689631,2.815220,10.000000
1,SVCJ,ell_y,-5.000000,5.000000,0.014706,0.000000,-5.000000,-0.148133,-0.051558,0.035857,0.893355
2,SVCJ,kappa,0.000100,50.000000,0.000000,0.227941,2.357031,7.089654,17.597731,44.222681,50.000000
3,SVCJ,lam,0.000001,10.000000,0.117647,0.000000,0.027555,0.837533,2.729907,3.515299,6.762650
4,SVCJ,rho,-0.999909,0.999909,0.205882,0.051471,-0.999756,-0.786318,-0.223090,0.144405,0.999909
5,SVCJ,rho_j,-0.999909,0.999909,0.154412,0.014706,-0.999909,-0.085847,-0.043150,-0.013870,0.999905
6,SVCJ,sigma_v,0.000100,10.000000,0.404412,0.000000,0.000113,0.025660,0.978035,4.060022,6.433524
7,SVCJ,sigma_y,0.000100,5.000000,0.463235,0.000000,0.000100,0.000232,0.112861,0.149514,1.973048
8,SVCJ,theta,0.000001,5.000000,0.117647,0.000000,0.000029,0.135123,0.211354,0.301843,0.420881
9,SVCJ,v0,0.000001,5.000000,0.029412,0.000000,0.044498,0.186135,0.220383,0.282074,2.223426


## 11) Export thesis artifacts (tables + figures)

This cell saves:
- tables into `./tables/`
- figures into `./figs/` as HTML (always) and PNG (if Kaleido is available)

Set `EXPORT = True` to activate.


In [15]:

EXPORT = False

OUT_TABLES = "tables"
OUT_FIGS = "figs"

def _safe_write_image(fig, path_png):
    try:
        fig.write_image(path_png, scale=2)
        return True
    except Exception as e:
        print(f"[warn] Could not write PNG (needs kaleido): {path_png}\n  {e}")
        return False

if EXPORT:
    os.makedirs(OUT_TABLES, exist_ok=True)
    os.makedirs(OUT_FIGS, exist_ok=True)

    # --- convergence ---
    conv = convergence_table(results_long)
    conv.to_csv(os.path.join(OUT_TABLES, "convergence_table.csv"), index=False)

    for currency in results_long["currency"].unique():
        for split in ["train","test"]:
            # error summary
            tbl = error_summary_table(results_long, currency, split=split)
            tbl.to_csv(os.path.join(OUT_TABLES, f"{currency.lower()}_{split}_error_summary.csv"), index=False)

            # spread summary
            tbl_sp = spread_metric_summary_table(opt_metrics, currency, split=split)
            tbl_sp.to_csv(os.path.join(OUT_TABLES, f"{currency.lower()}_{split}_spread_summary.csv"), index=False)

            # time series fig
            fig_ts = plot_error_timeseries(results_long, currency, split=split)
            fig_ts.write_html(os.path.join(OUT_FIGS, f"{currency.lower()}_{split}_errors_timeseries.html"))
            _safe_write_image(fig_ts, os.path.join(OUT_FIGS, f"{currency.lower()}_{split}_errors_timeseries.png"))

            # boxplots fig
            fig_box = plot_error_boxplots(results_long, currency, split=split)
            fig_box.write_html(os.path.join(OUT_FIGS, f"{currency.lower()}_{split}_errors_boxplots.html"))
            _safe_write_image(fig_box, os.path.join(OUT_FIGS, f"{currency.lower()}_{split}_errors_boxplots.png"))

            # spread fig
            fig_sp = spread_metric_timeseries(opt_metrics, currency, split=split)
            fig_sp.write_html(os.path.join(OUT_FIGS, f"{currency.lower()}_{split}_spread_timeseries.html"))
            _safe_write_image(fig_sp, os.path.join(OUT_FIGS, f"{currency.lower()}_{split}_spread_timeseries.png"))

        # buckets (test)
        b1 = bucket_summary_table(bucket_all, currency, "moneyness")
        b2 = bucket_summary_table(bucket_all, currency, "maturity")
        b1.to_csv(os.path.join(OUT_TABLES, f"{currency.lower()}_test_bucket_moneyness.csv"), index=False)
        b2.to_csv(os.path.join(OUT_TABLES, f"{currency.lower()}_test_bucket_maturity.csv"), index=False)

        fig_bm = plot_bucket_bars(bucket_all, currency, "moneyness")
        fig_bt = plot_bucket_bars(bucket_all, currency, "maturity")
        fig_bm.write_html(os.path.join(OUT_FIGS, f"{currency.lower()}_test_bucket_moneyness.html"))
        fig_bt.write_html(os.path.join(OUT_FIGS, f"{currency.lower()}_test_bucket_maturity.html"))
        _safe_write_image(fig_bm, os.path.join(OUT_FIGS, f"{currency.lower()}_test_bucket_moneyness.png"))
        _safe_write_image(fig_bt, os.path.join(OUT_FIGS, f"{currency.lower()}_test_bucket_maturity.png"))

        # bounds
        nb_hes = near_bound_rates(results_long[results_long["currency"]==currency], "Heston")
        nb_svcj = near_bound_rates(results_long[results_long["currency"]==currency], "SVCJ")
        nb_hes.to_csv(os.path.join(OUT_TABLES, f"{currency.lower()}_heston_near_bound_rates.csv"), index=False)
        nb_svcj.to_csv(os.path.join(OUT_TABLES, f"{currency.lower()}_svcj_near_bound_rates.csv"), index=False)

    print("Export complete.")
else:
    print("EXPORT=False (no files written). Set EXPORT=True to save tables/figures.")


EXPORT=False (no files written). Set EXPORT=True to save tables/figures.
